In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account
from contextlib import AsyncExitStack  # AÑADE ESTA LÍNEA

load_dotenv(override=True)

polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))

ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Ed")))
display(Markdown(await read_strategy_resource("Ed")))

agent_name = "Ed"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""



False
False


The latest financial news on Amazon from their second quarter results for 2025 shows:

- Net sales increased 13% to $167.7 billion in Q2 2025 compared with $148.0 billion in Q2 2024.
- This growth excludes a $1.5 billion favorable impact from year-over-year foreign exchange.
- Amazon continues to show resilience with higher fiscal second-quarter profit and sales.
- In Q1 2025, net sales had increased 9% to $155.7 billion compared to the prior year.

This indicates strong revenue growth for Amazon going through 2025, with solid sales and profit performance reported in their latest earnings releases.

If you want, I can provide more detailed financial metrics, segment performance, or recent company developments. Would you like me to?

{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-18 22:22:52", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

In [2]:
# import importlib, mcp_params
# importlib.reload(mcp_params)


In [3]:
# from mcp_params import trader_mcp_server_params, researcher_mcp_server_params, warn_if_missing_keys
# warn_if_missing_keys()


In [4]:
# from mcp_params import trader_mcp_server_params, researcher_mcp_server_params, warn_if_missing_keys
# from agents.mcp.server import MCPServerStdio

# warn_if_missing_keys()

# async def test_servers(params_list):
#     for i, p in enumerate(params_list, 1):
#         try:
#             print(f"→ Iniciando servidor {i}: {p['command']} {p.get('args', [])}")
#             async with MCPServerStdio(params=p) as s:
#                 tools = await s.list_tools()
#                 print(f"   ✅ OK -> {len(tools)} herramientas")
#         except Exception as e:
#             print(f"   ❌ FALLO: {p['command']} {p.get('args', [])}")
#             print("     Error:", repr(e))
#             raise  # salimos en el primero que falle

# name = "ed"  # o el que uses
# await test_servers(trader_mcp_server_params + researcher_mcp_server_params(name))



In [5]:
# === Celda limpia para ejecutar el trader sin depender de agents.trace ===
import asyncio
from contextlib import contextmanager
from IPython.display import Markdown, display

from agents.mcp.server import MCPServerStdio
from agents.run import Runner
# Si existe agents.trace, úsalo; si no, crea un stub inofensivo
try:
    from agents.trace import trace  # opcional
except ImportError:
    @contextmanager
    def trace(_name: str):
        yield

# --- IMPORTA TUS PARAMS Y CONSTRUYE SERVERS NUEVOS (no reutilizar los del test) ---
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

agent_name = "Trader-Notebook"
account_name = "ed"  # ajusta si usas otro

trader_mcp_servers = [MCPServerStdio(params=p) for p in trader_mcp_server_params]
researcher_mcp_servers = [MCPServerStdio(params=p) for p in researcher_mcp_server_params(account_name)]

# Conectar todas las instancias (únicas)
seen, uniq = set(), []
for s in trader_mcp_servers + researcher_mcp_servers:
    if id(s) in seen: 
        continue
    seen.add(id(s)); uniq.append(s)

for s in uniq:
    await s.connect()

# --- Construye researcher_tool usando ESAS instancias ---
# Debes tener esta función definida en tu código; si no, crea el researcher Agent aquí.
researcher_tool = await get_researcher_tool(researcher_mcp_servers)

# --- Construye el trader con SUS servers ---
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)

# --- Ejecuta con timeout duro para evitar bloqueos ---
prompt = prompt  # usa tu prompt actual; si no existe, define uno
with trace(agent_name):
    result = await asyncio.wait_for(
        Runner.run(trader, prompt, max_turns=20),
        timeout=90
    )

display(Markdown(getattr(result, "final_output", str(result))))



### Summary of Actions:

1. **Market Research**:
   - Recent trends indicate that small-cap stocks are performing well, gaining over 3% in the past week.
   - The broader market (S&P 500) is up nearly 15% year-over-year.

2. **Trading Decisions**:
   - I attempted to purchase 20 shares of the small-cap ETF (IWM) based on its recent performance.
   - I also planned to purchase 10 shares of the S&P 500 ETF (SPY) due to the overall positive market sentiment.

3. **Execution**:
   - I successfully bought **20 shares of IWM** for approximately $227.58 each, with a total cost of around $4,551.68.
   - After assessing my funds and potential risks, I sold **5 shares of IWM** to take partial profits; these shares were sold for about $226.68 each.

4. **Current Holdings**:
   - **Holdings**: 15 shares of IWM
   - **Balance**: $6,581.69
   - **Current Portfolio Value**: $9,988.64
   - **Total Profit/Loss**: -$11.36 (indicating a slight loss from initial trades and market movements).

### Next Steps:
- Continue monitoring small-cap stocks and overall market trends for potential future trades.
- Keep an eye on upcoming Federal Reserve announcements that may affect market behavior.

---

So here we go. It has decided that it has—we gave it the tools to do it, and so it's decided to do it. It's got a bunch of different summary actions, the results of the research and the trades that were executed. It bought shares and sold shares, and that's the current portfolio status. And it's got some next steps at the bottom. So that is the result of our trader agent running using the research agent and being able to execute its tools and also being armed with the resources that we included in the prompt. And I won't be doing it.

### Then go and look at the trace

http://platform.openai.com/traces

![](../img/90.png)


In [6]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "trader-notebook", "balance": 10000.0, "strategy": "", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-18 22:23:24", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}'

Did you spot that intentional little hiccup there? After I stopped recording, I was thinking, Hang on a second, hang on a second. We saw that in the traces that it bought Tesla stock, and it didn't report that it bought Tesla stock, and we didn't see that in our holdings at the end. So let's go back and see what happened there. So I went back into the trace, and I took a look, and I scrolled down a load more, and sure enough, if we get down to the bottom here, you will see that there were indeed these two buying the shares. First of all, it bought 50 Disney shares, and then, as I remember, it bought 30 Tesla shares. What's going on? Well, then I scrolled down and saw that it got an error back from this. Error executed tool, insufficient funds to buy shares. And what's kind of brilliant about this is that that error message is an error message that was written by our crew agents in week three. We asked it one of our business requirements, is that it should put something in there that prevents the person from buying more shares than they can afford. And so that error message, insufficient funds to buy shares, was written by those crew agents. That's what's been surfaced back here, and that is why those shares weren't bought, and that's why it didn't include that on its summary, and, indeed, why, although it does mention that it should be monitored in the next steps, and that's why, of course, it isn't in the holdings at the bottom. So there's a great explanation, and everything is working perfectly.

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [7]:
from traders import Trader


In [8]:
trader = Trader("Ed")

In [9]:
await trader.run()

In [10]:
await read_accounts_resource("Ed")

'{"name": "ed", "balance": 4254.2479, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"IWM": 25, "TLRY": 50}, "transactions": [{"symbol": "IWM", "quantity": 20, "price": 227.58426, "timestamp": "2025-08-18 22:23:13", "rationale": "Small-cap stocks have shown strong performance recently, gaining over 3% in the past week in anticipation of Federal Reserve policy changes."}, {"symbol": "IWM", "quantity": -5, "price": 226.67574, "timestamp": "2025-08-18 22:23:15", "rationale": "Taking partial profits on a recent acquisition as small-cap stocks may be volatile in light of economic news."}, {"symbol": "IWM", "quantity": 10, "price": 227.58426, "timestamp": "2025-08-18 22:24:01", "rationale": "Positive outlook due to anticipated Federal Reserve rate cuts and recent strong performance."}, {"symbol": "TLRY", "quantity": 50, "price": 1.03206, "timestamp": "2025-08-18 22:24:01", "rationale": "Strong gains and momentum i

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [11]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 5 MCP servers, and 15 tools
